<a href="https://colab.research.google.com/github/add-itya/SpotifyFilter/blob/main/spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy


In [ ]:
import json
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

username = "___"
client_id = "___"
client_secret = "___"
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)


##oauth
scope = "playlist-modify-public playlist-read-private playlist-modify-private"
sp = spotipy.Spotify(
        auth_manager=spotipy.SpotifyOAuth(
          client_id=client_id,
          client_secret=client_secret,
          redirect_uri= "https://example.com/callback/",    
          scope= scope, open_browser=False))

def playlist_features(track_ids): 
  p_feat_list = []
  for i in range(len(track_ids)):
    p_feat_list.append(sp.audio_features(track_ids[i]))
  return p_feat_list


#name of function lol
def list_of_tracks_and_playlist_id(category, lower, upper, features_list):
  okay_tracks = []
  for i in range(len(features_list)):
    category_val = features_list[i][0][category]
    if category_val >= lower and category_val <= upper:
      okay_tracks.append(features_list[i][0]["id"])
  return okay_tracks


#functions to make playlist
import math
def make_playlist(trackids, playlist_name, description):
  l = len(trackids)
  n = math.ceil(l/100)
  a = sp.user_playlist_create("21wfyastuderpsx3drltmt52i", name = playlist_name, public = False, collaborative = False, description = description)
  playlist_id = a["id"]
  for i in range(n):
    helper(trackids[i*100:100*(i+1)], playlist_id)
    
def helper(trackids, playlist_id):
  sp.playlist_add_items(playlist_id, trackids)



#functions to read playlist

def show_tracks(results, uriArray):
  for i, item in enumerate(results["items"]):
    track = item['track']
    uriArray.append(track["id"])

def get_playlist_track_id(username, playlist_id):
  trackID = []
  results = sp.user_playlist(username, playlist_id)
  tracks = results["tracks"]
  show_tracks(tracks, trackID)
  while tracks["next"]:
      tracks = sp.next(tracks)
      show_tracks(tracks, trackID)
  return trackID



#function that gets playlist id from name
a = sp.current_user_playlists()
def get_preexisting_id(name):
  id = ""
  for i in a["items"]:
    if i["name"] == name:
      id = i["id"]
  return id

#implementation

#6 different categories:

#Acousticness
#danceability
#instrumentalness
#liveness
#speechiness
#valence

#have the list of trackids
#input: username and playlist id
mango_lassi_track_ids = get_playlist_track_id(username, "02o7Vidb51HA7cX6CfCzx8?si=a7b9c6acf59449b3")

#playlist features
mango_lassi_features_list = playlist_features(mango_lassi_track_ids)

#create featured playlist
#input category, range
trackids_filtered = list_of_tracks_and_playlist_id("danceability", 0.8, 1.0, mango_lassi_features_list)


make_playlist(trackids_filtered, "gym shi", "oops")